In [14]:
import numpy as np
import keras as K
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM

import numpy as np

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

print("Sentiment Analysis on IMDB data set with keras")

Sentiment Analysis on IMDB data set with keras


# Load Data ...



In [16]:
max_words= 25000
print("Loading Data .....")
(train_x, train_y), (test_x,test_y)= K.datasets.imdb.load_data(num_words=max_words)

print(train_x[0:5])
print(train_y[0:5])

Loading Data .....


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 7

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


# Pre Processing

In [18]:
max_review_length=160
train_x=K.preprocessing.sequence.pad_sequences(train_x, truncating='pre',
                                               padding='pre',maxlen=max_review_length)
test_x=K.preprocessing.sequence.pad_sequences(test_x, truncating='pre',
                                               padding='pre',maxlen=max_review_length)
print(train_x[0:5])


[[   14    22     4  1920  4613   469     4    22    71    87    12    16
     43   530    38    76    15    13  1247     4    22    17   515    17
     12    16   626    18 19193     5    62   386    12     8   316     8
    106     5     4  2223  5244    16   480    66  3785    33     4   130
     12    16    38   619     5    25   124    51    36   135    48    25
   1415    33     6    22    12   215    28    77    52     5    14   407
     16    82 10311     8     4   107   117  5952    15   256     4     2
      7  3766     5   723    36    71    43   530   476    26   400   317
     46     7     4 12118  1029    13   104    88     4   381    15   297
     98    32  2071    56    26   141     6   194  7486    18     4   226
     22    21   134   476    26   480     5   144    30  5535    18    51
     36    28   224    92    25   104     4   226    65    16    38  1334
     88    12    16   283     5    16  4472   113   103    32    15    16
   5345    19   178    32]
 [  110  31

# Define the MODEL


In [19]:
embedding_length = 64
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_length, input_length=max_review_length))
model.add(LSTM(units=64, input_shape=(max_review_length, embedding_length), return_sequences=False, unroll=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 160, 64)           1600000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,633,089
Trainable params: 1,633,089
Non-trainable params: 0
_________________________________________________________________


# Train the model

In [20]:
print('Training...')
batch_size = 32
model.fit(train_x, train_y,
          batch_size = batch_size,
          epochs=10,
          validation_data = (test_x, test_y))


Training...
Epoch 1/10
782/782 [==============================] - 88s 85ms/step - loss: 0.5132 - accuracy: 0.7256 - val_loss: 0.3242 - val_accuracy: 0.8679
Epoch 2/10
782/782 [==============================] - 64s 82ms/step - loss: 0.1956 - accuracy: 0.9257 - val_loss: 0.3644 - val_accuracy: 0.8561
Epoch 3/10
782/782 [==============================] - 64s 82ms/step - loss: 0.1168 - accuracy: 0.9610 - val_loss: 0.3882 - val_accuracy: 0.8487
Epoch 4/10
782/782 [==============================] - 64s 82ms/step - loss: 0.0816 - accuracy: 0.9734 - val_loss: 0.4693 - val_accuracy: 0.8566
Epoch 5/10
782/782 [==============================] - 64s 82ms/step - loss: 0.0477 - accuracy: 0.9857 - val_loss: 0.5772 - val_accuracy: 0.8445
Epoch 6/10
782/782 [==============================] - 64s 82ms/step - loss: 0.0422 - accuracy: 0.9848 - val_loss: 0.6135 - val_accuracy: 0.8504
Epoch 7/10
782/782 [==============================] - 64s 82ms/step - loss: 0.0266 - accuracy: 0.9912 - val_loss: 0.6299 - v

# Evaluate the model

In [21]:
score, acc = model.evaluate(test_x,test_y, batch_size=batch_size)
print(f'Test score = {score}')
print(f'Test accuracy = {acc}')
model.save('/content/drive/MyDrive/Caption/model.h5')

782/782 [==============================] - 12s 16ms/step - loss: 0.7505 - accuracy: 0.8470
Test score = 0.7504640817642212
Test accuracy = 0.8469600081443787


# Making predictions with model

In [22]:
review = "I love cat."
print(f'New review = {review}')

d = imdb.get_word_index()
words = review.split()
review = []

for word in words:
  if word not in d:
    review.append(2)
  else:
    review.append(d[word] + 3)
  
print(f"review = {review}")

review = sequence.pad_sequences([review], truncating='pre', padding='pre', maxlen=80)

prediction = model.predict(review)
print(f'Prediction (0 = Negative, 1 = positive) = {prediction}')

New review = I love cat.
1646592/1641221 [==============================] - 0s 0us/step
review = [2, 119, 2]
Prediction (0 = Negative, 1 = positive) = [[0.88790345]]
